In [1]:
# Importación de librerias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import time
import requests
import cv2
import urllib
import numpy as np

In [2]:
class Page(object):
    def __init__(self, driver):
        self.driver = driver
        self.wait = WebDriverWait(driver, WAIT_TIME)

class SearchPage(Page):
    def get_images_URLs(self):
        global total
        URLs = []
        i = 0
        image_tags = self.driver.find_elements_by_css_selector("img.z_h_9d80b.z_h_2f2f0")
            
        while(total < amt and i < len(image_tags)):
            src = image_tags[i].get_attribute("src")
            URLs.append(src)
            i = i + 1
            total = total + 1
            print("Found image with src: ", src)
            print("Total: ", total)
        return URLs
        
    def scroll_down(self):
        print("Scrolling down...")
        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") 
        # Wait 4 seconds for all the images to load
        self.wait.until(lambda d: d.find_element_by_css_selector("a[data-automation='BottomNav_NextButton']"))
        print("Waiting for images to load...")
        time.sleep(5)
        
        
    def go_next(self):
        print("Changing page...")
        next_btn = self.driver.find_element_by_css_selector("a[data-automation='BottomNav_NextButton']");
        driver.execute_script("arguments[0].click();", next_btn)
        #next_btn.click();
        print("Waiting for images to load...")
        time.sleep(5)
        print(self.driver.title)
    
class MainPage(Page):
    def search_term(self, term):
        elem = self.driver.find_element_by_name("searchterm")
        elem.clear()
        elem.send_keys(term)
        elem.send_keys(Keys.RETURN)
        self.wait.until(lambda d: term in d.title.lower())


total = 0
WAIT_TIME = 30
URLs = []
term = input("Introduce el término de búsqueda: ")
term = term.lower()
amt = input("Introduce la cantidad de imágenes que deseas obtener: ")
amt = int(amt)

with webdriver.Firefox() as driver:
    driver.maximize_window()
    driver.get("https://www.shuttersotck.com")
    main_page = MainPage(driver)
    main_page.search_term(term)
    
    
    search_page = SearchPage(driver)
    search_page.scroll_down()
    while((page_images_URLs := search_page.get_images_URLs()) and len(URLs) < amt):
        URLs.extend(page_images_URLs)
        search_page.go_next()
        search_page.scroll_down()
    print(URLs)
    print("Total number: ", len(URLs))

Introduce el término de búsqueda: dogs
Introduce la cantidad de imágenes que deseas obtener: 100
Scrolling down...
Waiting for images to load...
Found image with src:  https://image.shutterstock.com/image-photo/row-tops-heads-cats-dogs-260nw-1034939470.jpg
Total:  1
Found image with src:  https://image.shutterstock.com/image-photo/cropped-image-handsome-young-man-260nw-708732331.jpg
Total:  2
Found image with src:  https://image.shutterstock.com/image-photo/little-chihuahuas-spitz-front-white-260nw-1686200872.jpg
Total:  3
Found image with src:  https://image.shutterstock.com/image-photo/happy-puppy-dog-smiling-on-260nw-1799966587.jpg
Total:  4
Found image with src:  https://image.shutterstock.com/image-vector/vector-illustration-dogs-different-breeds-260nw-1708437577.jpg
Total:  5
Found image with src:  https://image.shutterstock.com/image-photo/pure-youth-crazy-english-cocker-260nw-1424153078.jpg
Total:  6
Found image with src:  https://image.shutterstock.com/image-photo/broun-dog-da

Found image with src:  https://image.shutterstock.com/image-photo/couple-dogs-love-sleeping-together-260nw-777863431.jpg
Total:  81
Found image with src:  https://image.shutterstock.com/image-vector/paw-vector-foot-trail-print-260nw-1140245744.jpg
Total:  82
Found image with src:  https://image.shutterstock.com/image-photo/small-dog-goes-walk-260nw-1554764084.jpg
Total:  83
Found image with src:  https://image.shutterstock.com/image-photo/golden-retriever-looking-out-car-260nw-407893192.jpg
Total:  84
Found image with src:  https://image.shutterstock.com/image-photo/dogs-isolated-on-white-260nw-333646538.jpg
Total:  85
Found image with src:  https://image.shutterstock.com/image-vector/set-creating-dog-animation-emotions-260nw-744646942.jpg
Total:  86
Found image with src:  https://image.shutterstock.com/image-vector/dog-paw-isolated-bone-seamless-260nw-1021326958.jpg
Total:  87
Found image with src:  https://image.shutterstock.com/image-photo/happy-young-man-holding-dog-260nw-582115876

In [3]:
def url2img(link):
    '''
    :param link: URLs de imagenes recolectadas.
    :return: imagen correspondiente al link proporcionado. 
    '''
    with urllib.request.urlopen(link) as link:
        source = link.read()
    img = np.asarray(bytearray(source), dtype="uint8")
    img = cv2.imdecode(img, cv2.IMREAD_COLOR)

    return img

In [4]:
n_train = int(len(URLs)*0.8)
n_test = len(URLs) - n_train

In [5]:
train_URLs = []
test_URLs = []
itr = 0
for n in URLs:
    if itr < n_train:
        train_URLs.append(n)
    else:
        test_URLs.append(n)
    itr += 1

In [6]:
def img_transfer(links,path):
    '''
    :param links: links separados para folder train o test.
    :param path: path dir.
    :return: guardado de las imagenes en sus respectivos folders.
    '''
    itr = 0
    for m in links:
        img = url2img(m) 
        img_name = term + str(itr) + '.jpg'
        cv2.imwrite(os.path.join(path, img_name), img)
        itr += 1
    return

In [7]:
# Recolectando imagenes para train y test
path1 = r'train\\' + term
path2 = r'test\\' + term
try:
    os.makedirs(path1)
    os.makedirs(path2)
except:
    print("Directory " , path1 , "or ", path2, " already exists")
img_transfer(train_URLs,path1)
img_transfer(test_URLs,path2)

In [8]:
URLs

['https://image.shutterstock.com/image-photo/row-tops-heads-cats-dogs-260nw-1034939470.jpg',
 'https://image.shutterstock.com/image-photo/cropped-image-handsome-young-man-260nw-708732331.jpg',
 'https://image.shutterstock.com/image-photo/little-chihuahuas-spitz-front-white-260nw-1686200872.jpg',
 'https://image.shutterstock.com/image-photo/happy-puppy-dog-smiling-on-260nw-1799966587.jpg',
 'https://image.shutterstock.com/image-vector/vector-illustration-dogs-different-breeds-260nw-1708437577.jpg',
 'https://image.shutterstock.com/image-photo/pure-youth-crazy-english-cocker-260nw-1424153078.jpg',
 'https://image.shutterstock.com/image-photo/broun-dog-dachshund-sun-glasses-260nw-1660171375.jpg',
 'https://image.shutterstock.com/image-photo/funny-studio-portrait-smilling-puppy-260nw-1008869716.jpg',
 'https://image.shutterstock.com/image-photo/dog-260nw-587562362.jpg',
 'https://image.shutterstock.com/image-vector/dogs-collection-vector-illustration-funny-260nw-740988544.jpg',
 'https://i